### Carlos Tardón y Carlos Morán, grupo 9

## Tercera parte.  Recomendacion basada en filtrado colaborativo.

En esta tercera parte utilizaremos la librería SURPRISE Se puede consultar la documentacion en http://surpriselib.com/

Para instalarla: conda install -c conda-forge scikit-surprise o pip install numpy pip install scikit-surprise

La librería SurPRISE (Simple Python RecommendatIon System Engine) tiene algoritmos de predición de ratings incluidos: baseline algorithms, neighborhood methods, matrix factorization-based ( SVD, PMF, SVD++, NMF) y otros. También tiene predefinidas las medidas de similitud mas comunes sobre vectores (cosine, MSD, pearson…) Una de las cosas más utiles es que proporciona herramientas para evaluar, analizar y comparar el rendimiento de distitnos algoritmos. Lo que vamos a hacer en esta parte de la práctica es probar varios procedimientos de evaluación cruzada midiendo datos sobre errores entre el valor real (conocido) y la predicción del recomendador. Las siglas corresponden a las siguientes medidas:

MAE: Mean Absolute Error
RMSE: Root mean square error (RMSE)
MSE: mean square error is defined as the expected value of the square of the difference between the estimator and the parameter. -square root of the mean square error.

Vamos a ejecutar algunos recomendadores y evaluarlos para poder comentar los resultados.


In [1]:
from collections import defaultdict
import numpy as np

from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline

from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
## Ejemplo getting started de la documentación de SURPRISE
##http://surpriselib.com/

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)



Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9371  0.9301  0.9338  0.9282  0.9397  0.9338  0.0043  
MAE (testset)     0.7367  0.7358  0.7355  0.7314  0.7398  0.7358  0.0027  
Fit time          11.65   11.27   11.33   11.56   11.30   11.43   0.15    
Test time         0.48    0.45    0.36    0.43    0.36    0.42    0.05    


{'test_rmse': array([0.93705508, 0.93007129, 0.93383281, 0.92821858, 0.93970594]),
 'test_mae': array([0.73665742, 0.73577396, 0.73552484, 0.7313606 , 0.73977581]),
 'fit_time': (11.65084719657898,
  11.273847818374634,
  11.332695245742798,
  11.56409764289856,
  11.30376672744751),
 'test_time': (0.4767286777496338,
  0.45079898834228516,
  0.35704588890075684,
  0.43282246589660645,
  0.36103367805480957)}

In [3]:
# Evaluacion extracted from surprise: 
# https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-compute-precision-k-and-recall-k
def measures_at_k(predictions, k, th_recom, th_relev):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    onehits = dict()
    mrr = dict()
    
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= th_relev) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= th_recom) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= th_relev) and (est >= th_recom))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
       
        
    return precisions, recalls


In [4]:
def f1(precision, recall):
    """
        Funcion que calcula el f1 (media armónica) en funcion de precision y recall
    """
    denominador = precision + recall
    
    if denominador == 0:
        return 0
    else:
        return (2 * precision * recall) / denominador

In [5]:
def get_results(recommendations, k, knn):
    """
        Function to get the measures results 
    """
    # threshold = 4 --> solo se tienen en cuenta peliculas que hayan 
    # sido puntuadas con 4 o 5 estrellas
    precisions, recalls  = measures_at_k(recommendations, k, th_recom=4, th_relev=1)
    
    # Measures can then be averaged over all users
    precision_result = sum(prec for prec in precisions.values()) / len(precisions)
    recall_result = sum(rec for rec in recalls.values()) / len(recalls)
    # Media armónica  
    f1_result = f1(precision_result, recall_result)
    # En este archivo se volcarán los resultados de las métricas. Tiene que existir. 
    f = open("results_user_cf.csv", 'a')
    #f = open("C:/hlocal/results_user_cf.csv", 'a')
    f.write(str(k) + ',' + knn + "," + str(precision_result) + ',' + str(recall_result) + ',' +  str(f1_result) +  '\n') 
    f.close()
    

In [ ]:
# Hemos cargado antes los datos de movieLens para 100K
# data = Dataset.load_builtin('ml-100k')

In [6]:
# creo dos conjuntos de datos, el training set y el evaluation set
# cada uno contendra la mitad de los datos
training_set, evaluation_set = train_test_split(data, test_size=.5)

In [7]:
# Ahora determino cual es el algoritmo que voy a usar de recomendacion
# en este caso voy a usar el algoritmo KNN para encontrar las similitudes entre items
recommendation_algorithm = KNNBasic(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})
#print(recommendation_algorithm)

In [8]:
# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)
#print(recommendation_algorithm)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [9]:
# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)
#print(recommendations)

In [11]:
K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_basic")

In [12]:
##########################################################
# Hacer distintas pruebas con el resto de tipos KNN
recommendation_algorithm = KNNWithMeans(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withmeans")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [13]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNWithZScore(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_withzscore")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [14]:
##########################################################
# Hago lo mismo con el resto de tipos KNN
recommendation_algorithm = KNNBaseline(k=100, sim_options={'name': 'pearson_baseline', 'user_based': True})

# aplico el algoritmo sobre el training_set
recommendation_algorithm.fit(training_set)

# aplico el algoritmo sobre el evaluation set y obtengo las predicciones en las recomendaciones
recommendations = recommendation_algorithm.test(evaluation_set)

K = 10
for k in range(K):
    get_results(recommendations, k+1, "knn_baseline")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


## Ejercicio:  se pide ejecutar, comprender y escribir comentarios razonados sobre la evaluación de distintos recomendadores.
    
Prueba otros algoritmos de predicción de ratings basados en la estimación de los vecinos más próximos y realiza evaluaciones de su comportamiento. Comenta los resultados.¶
Puedes consultar la documentación en https://surprise.readthedocs.io/en/stable/knn_inspired.html#


#### Explicación de cada recomendador
Los 4 recomendadores que aquí estudiamos (KNNBasic, KNNWithZScore, KNNWithMeans, KNNBaseline) se derivan del enfoque KNN, donde la estimación del recomendador se calcula agregando los valores de los k vecinos más próximos. La diferencia entre estos 4 estimadores está en la forma de agregar estas valoraciones.

En el KNNBasic, los valores se agregan haciendo una media ponderada por la similitud. Por tanto, los vecinos más similares(más cercanos) tendrán una mayor influencia en el resultado final. Destaca por su sencillez.

En KNNWithMeans, se hace una media ponderada por la similitud, pero no de los valores de los vecinos, sino de las distancias entre esos valores y su media.  

En KNNWithZScore se hace una media ponderada de los z-score, es decir, de la unidad tipificada(restando la media y dividiendo entre la desviación típica)

Por último, KNNBaseline es una aproximación parecida a KNNWithMeans, pero en vez de la media usa un _baseline_ con el objetivo de mejorar la recomendación.

#### Comparemos el comportamiento de los algoritmos
Primero, realizaremos una validación cruzada con las medidas RMSE y MAE: 

In [10]:
algo = KNNBasic(k=100, verbose=False, sim_options={'name': 'pearson_baseline', 'user_based': True})
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9971  1.0041  1.0053  1.0015  0.9951  1.0006  0.0040  
MAE (testset)     0.7878  0.7920  0.7949  0.7945  0.7883  0.7915  0.0030  
Fit time          1.33    1.19    1.60    1.56    1.57    1.45    0.16    
Test time         4.68    5.12    5.97    6.05    5.92    5.55    0.55    


{'test_rmse': array([0.99706653, 1.00414383, 1.0053323 , 1.0014597 , 0.99506959]),
 'test_mae': array([0.78782037, 0.79204042, 0.79488668, 0.79450741, 0.78830258]),
 'fit_time': (1.3285479545593262,
  1.1863369941711426,
  1.602675199508667,
  1.5630590915679932,
  1.5683982372283936),
 'test_time': (4.680873155593872,
  5.116900205612183,
  5.971094369888306,
  6.04677152633667,
  5.9150230884552)}

In [11]:
algo = KNNWithZScore(k=100, verbose=False, sim_options={'name': 'pearson_baseline', 'user_based': True})
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm KNNWithZScore on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9410  0.9323  0.9453  0.9379  0.9313  0.9376  0.0053  
MAE (testset)     0.7308  0.7261  0.7350  0.7308  0.7256  0.7297  0.0035  
Fit time          1.85    1.81    1.74    1.80    1.67    1.78    0.06    
Test time         7.04    6.85    6.95    7.11    7.21    7.03    0.12    


{'test_rmse': array([0.94101757, 0.93232102, 0.9452835 , 0.93791477, 0.93131122]),
 'test_mae': array([0.73075351, 0.72609902, 0.7350499 , 0.73079929, 0.72560364]),
 'fit_time': (1.8545176982879639,
  1.813765048980713,
  1.740574598312378,
  1.8017897605895996,
  1.6682560443878174),
 'test_time': (7.0405566692352295,
  6.853313207626343,
  6.949240684509277,
  7.106757402420044,
  7.210302114486694)}

In [12]:
# Probamos con la similitud del coseno. Empeora el error cuadratico medio
algo = KNNWithZScore(k=100, verbose=False, sim_options={'name': 'cosine', 'user_based': True})
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm KNNWithZScore on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9479  0.9561  0.9473  0.9553  0.9569  0.9527  0.0042  
MAE (testset)     0.7450  0.7508  0.7440  0.7511  0.7507  0.7483  0.0032  
Fit time          1.37    1.42    1.50    1.47    1.38    1.43    0.05    
Test time         7.28    7.40    7.85    7.35    7.53    7.48    0.20    


{'test_rmse': array([0.94793427, 0.95611606, 0.94730487, 0.95531864, 0.95685222]),
 'test_mae': array([0.74503644, 0.7508304 , 0.74395504, 0.75112569, 0.75074185]),
 'fit_time': (1.365238904953003,
  1.415797233581543,
  1.4987385272979736,
  1.4682555198669434,
  1.3806555271148682),
 'test_time': (7.281141042709351,
  7.4013354778289795,
  7.84873628616333,
  7.350979328155518,
  7.529534578323364)}

In [13]:
algo = KNNWithMeans(k=100, verbose=False, sim_options={'name': 'cosine', 'user_based': True})
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9570  0.9502  0.9548  0.9560  0.9583  0.9553  0.0028  
MAE (testset)     0.7562  0.7462  0.7556  0.7507  0.7571  0.7531  0.0041  
Fit time          1.39    1.54    1.30    1.46    1.30    1.40    0.09    
Test time         6.67    7.40    6.79    6.55    6.67    6.82    0.30    


{'test_rmse': array([0.95703715, 0.95022226, 0.95480904, 0.95603234, 0.95834078]),
 'test_mae': array([0.75616416, 0.74617468, 0.75556051, 0.75065941, 0.75705995]),
 'fit_time': (1.391956090927124,
  1.5360965728759766,
  1.2973341941833496,
  1.4586527347564697,
  1.2995209693908691),
 'test_time': (6.671422004699707,
  7.404827356338501,
  6.7909135818481445,
  6.548822641372681,
  6.66856837272644)}

In [14]:
#Probamos el último tipo de KNN. Es el que mejor error cuadrático medio posee, con RMSE 0.9216 y MAE 0.7223
algo = KNNBaseline(k=100, verbose=False, sim_options={'name': 'pearson_baseline', 'user_based': True})
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9211  0.9246  0.9271  0.9188  0.9167  0.9216  0.0038  
MAE (testset)     0.7223  0.7241  0.7270  0.7191  0.7189  0.7223  0.0031  
Fit time          1.52    1.54    1.73    1.69    1.58    1.61    0.08    
Test time         7.67    7.85    7.66    7.24    7.74    7.64    0.21    


{'test_rmse': array([0.9211128 , 0.92456164, 0.92705595, 0.91878244, 0.91671799]),
 'test_mae': array([0.72226533, 0.72408205, 0.72696128, 0.71908342, 0.71890669]),
 'fit_time': (1.5238189697265625,
  1.5369915962219238,
  1.7276537418365479,
  1.6889476776123047,
  1.57944655418396),
 'test_time': (7.6736791133880615,
  7.854694604873657,
  7.662973642349243,
  7.241253852844238,
  7.742943048477173)}

#### Resultados obtenidos

A continuación ofrecemos los datos extraídos del fichero csv. 

<table style="width:80%">
<tr border="bottom">
<td align="center">Algoritmo</td>
<td align="center">k</td>
<td align="center">Precisión</td>
<td align="center">Recall</td>
<td align="center">f1</td>
</tr>
<tr>
<td rowspan="3"> KNNBasic </td>
    <td>1</td>
<td>0.9565217391304348</td>
<td>0.03736769319682807</td>
<td>0.07192552757146778</td>
</tr>
<tr>
    <td>6</td>
<td>0.9565217391304348</td>
<td>0.12516194862022245</td>
<td>0.22135884292777913</td>
</tr>
    <tr>
    <td>10</td>
<td>0.9565217391304348</td>
<td>0.1757298050936013</td>
<td>0.2969117236230051</td>
</tr>
<tr>
<td colspan="5", style="height:50px"></td>
    </tr>
<tr>
<td rowspan="3"> KNN_withmeans </td>
  <td>1</td>
<td>0.8324496288441146</td>
<td>0.03173731545381958</td>
<td>0.06114352142059743</td>
</tr>
<tr>
    <td>6</td>
<td>0.8324496288441146</td>
<td>0.12438017987038016</td>
<td>0.21642351361893494</td>
</tr>
    <tr>
    <td>10</td>
<td>0.8324496288441146</td>
<td>0.19618169995117293</td>
<td>0.3175314201281977</td>
</tr>
<tr>
<td colspan="5", style="height:50px"></td>
</tr>
<tr>
<td rowspan="3"> KNN_withzscore </td>
    <td>1</td>
<td>0.8472958642629904</td>
<td>0.03222569512002612</td>
<td>0.06208988945615473</td>
</tr>
<tr>
    <td>6</td>
<td>0.8472958642629904</td>
<td>0.12460074322957036</td>
<td>0.21725293330302703</td>
</tr>
    <tr>
    <td>10</td>
<td>0.8472958642629904</td>
<td>0.19663515433795892</td>
<td>0.31919379742648424</td>
</tr>
<tr>
<td colspan="5", style="height:50px"></td>
</tr>
<tr>
<td rowspan="3"> KNN_baseline </td>
     <td>1</td>
<td>0.9257688229056203</td>
<td>0.03694134380519141</td>
<td>0.07104764352479713</td>
</tr>
<tr>
     <td>6</td>
<td>0.9257688229056203</td>
<td>0.13287370326098252</td>
<td>0.23239257600666582</td>
</tr>
    <tr>
     <td>10</td>
<td>0.9257688229056203</td>
<td>0.19126218212339602</td>
<td>0.31702712711387493</td>
</tr>
</table>


#### Interpretación y comparación

Lo primero que observamos es la diferencia de valores entre Precision y Recall. Mientras que Precision se mantiene siempre por encima de 0.83, recall no supera el 0.15 en ningún caso. Este comportamiento es esperable. En un recomendador, nos interesa que las películas que recomendamos al usuario le gusten lo más posible, y es exactamente esto lo que mide la métrica precisión, es decir, la fracción de las películas recomendadas que son relevantes. En cambio, la métrica recall mide la fracción de items relevantes que son recomendados. Esta métrica no es tan relevante en este caso, pues mejorarla solo aumentaría el número total de recomendaciones que podemos darle al usuario. Por tanto, con que el recall se mantenga por encima de un umbral es suficiente.

Como vemos, los algoritmos que otorgan mejores resultados en cuanto a precisión y recall son knn_basic y knn_baseline. Además, knn_baseline era el que minimizaba las medidas 'RMSE'y 'MAE', así que nos quedamos con knn_baseline para hacer comparaciones con el resto de recomendadores.

### SVD

In [10]:
# El algoritmo SVD,que está basado en factorización de matrices
algo = SVD() # Creamos el objeto svd, con parámetros por defecto

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9386  0.9404  0.9386  0.9305  0.9385  0.9373  0.0035  
MAE (testset)     0.7384  0.7409  0.7403  0.7332  0.7393  0.7384  0.0028  
Fit time          11.51   11.53   10.93   10.91   10.96   11.17   0.29    
Test time         0.38    0.44    0.43    0.34    0.43    0.40    0.04    


{'test_rmse': array([0.93855379, 0.9403645 , 0.93861045, 0.93045034, 0.93853057]),
 'test_mae': array([0.73843435, 0.74090468, 0.7403104 , 0.73318345, 0.7393197 ]),
 'fit_time': (11.508193492889404,
  11.527180194854736,
  10.928775310516357,
  10.90683913230896,
  10.960696458816528),
 'test_time': (0.3819754123687744,
  0.44081854820251465,
  0.4298527240753174,
  0.3390960693359375,
  0.4308464527130127)}

In [11]:
training_set, evaluation_set = train_test_split(data, test_size=.5)
algo.fit(training_set)

recommendations = algo.test(evaluation_set)

for k in [1,6,10]:
    get_results(recommendations, k, "SVD")

### Slope One

In [12]:
from surprise.prediction_algorithms.slope_one import SlopeOne

algo = SlopeOne()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9471  0.9403  0.9549  0.9378  0.9426  0.9445  0.0060  
MAE (testset)     0.7461  0.7362  0.7488  0.7376  0.7442  0.7426  0.0049  
Fit time          1.76    1.83    1.89    1.81    1.91    1.84    0.05    
Test time         6.88    7.27    7.46    7.30    7.39    7.26    0.20    


{'test_rmse': array([0.94707839, 0.94031413, 0.95489261, 0.93775036, 0.94259978]),
 'test_mae': array([0.7460805 , 0.73615954, 0.74880279, 0.7376372 , 0.74423424]),
 'fit_time': (1.7602977752685547,
  1.8261158466339111,
  1.885951042175293,
  1.8131506443023682,
  1.9138834476470947),
 'test_time': (6.875615835189819,
  7.267587184906006,
  7.4590537548065186,
  7.298485517501831,
  7.389246463775635)}

In [13]:
algo.fit(training_set)

recommendations = algo.test(evaluation_set)

for k in [1,6,10]:
    get_results(recommendations, k, "SlopeOne")

### NMF

In [14]:
from surprise import NMF
# Algoritmo parecido a SVD. También basado en el producto de matrices
algorithm = NMF()
cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9675  0.9591  0.9636  0.9677  0.9573  0.9630  0.0043  
MAE (testset)     0.7609  0.7536  0.7577  0.7582  0.7526  0.7566  0.0031  
Fit time          13.28   13.37   13.40   13.21   13.38   13.33   0.07    
Test time         0.44    0.30    0.43    0.28    0.39    0.37    0.07    


{'test_rmse': array([0.96754163, 0.95907227, 0.96358748, 0.96766932, 0.95732019]),
 'test_mae': array([0.76094016, 0.75355091, 0.75773667, 0.75815169, 0.75261421]),
 'fit_time': (13.28148341178894,
  13.370248794555664,
  13.400166511535645,
  13.210677862167358,
  13.380219459533691),
 'test_time': (0.4418184757232666,
  0.3041844367980957,
  0.42586374282836914,
  0.275266170501709,
  0.38696861267089844)}

In [16]:
algorithm.fit(training_set)

recommendations = algorithm.test(evaluation_set)

for k in [1,6,10]:
    get_results(recommendations, k, "NMF")

#### Resultados obtenidos

A continuación ofrecemos los datos extraídos del fichero csv:

<table style="width:75%">
<tr border="bottom">
<td align="center">Algoritmo</td>
<td align="center">k</td>
<td align="center">Precisión</td>
<td align="center">Recall</td>
<td align="center">f1</td>
</tr>
<tr>
<td rowspan="3"> SVD </td>
    <td>1</td>
<td>0.8441145281018028</td>
<td>0.03169942129583380</td>
<td>0.0611041695936367</td>
</tr>
<tr>
    <td>6</td>
<td>0.8441145281018028</td>
<td>0.12788856550036207</td>
<td>0.2221239764101800</td>
</tr>
<tr>
    <td>10</td>
<td>0.8441145281018028</td>
<td>0.16773051771603772</td>
<td>0.2798526659696334</td>
</tr>
<tr>
<td colspan="4", style="height:50px"></td>
</tr>
<tr>
<td rowspan="3"> Slope One </td>
    <td>1</td>
<td>0.8324496288441146</td>
<td>0.03048242326547522</td>
<td>0.05881130935299061</td>
</tr>
<tr>
    <td>6</td>
<td>0.8324496288441146</td>
<td>0.13285433299852145</td>
<td>0.22913930651199530</td>
</tr>
<tr>
    <td>10</td>
<td>0.8324496288441146</td>
<td>0.17973879658309724</td>
<td>0.29564355953060006</td>
</tr>
<tr>
<td colspan="4", style="height:50px"></td>
</tr>
<tr>
<td rowspan="3"> NMF </td>
    <td>1</td>
<td>0.8706256627783670</td>
<td>0.03338207198485823</td>
<td>0.06429876079399810</td>
</tr>
<tr>
    <td>6</td>
<td>0.8706256627783670</td>
<td>0.14204241678755442</td>
<td>0.24423752610295515</td>
</tr>
<tr>
    <td>10</td>
<td>0.8706256627783670</td>
<td>0.18431278701594284</td>
<td>0.30422143090067655</td>
</tr>
</table>


#### Interpretación y comparación

Vemos que, tanto las métricas 'RMSE' y 'MAE', como la Precisión y el Recall son ligeramente peores a las obtenidas con KNN_Baseline en los casos del SVD y SlopeOne. Por otro lado, aunque la precisión mejora ligeramente en el caso del NMF, empeoran las métricas 'RMSE' y 'MAE' con respecto a los dos anteriores. Deducimos que NMF empeora al SVD y, por lo tanto, al KNN_Baseline